In [1]:
import numpy as np
import inspect
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

from math import exp, sqrt
from sympy import *

# Лабораторная №2

## Utils

### Полезные ссылки

- [условие](http://mathdep.ifmo.ru/wp-content/uploads/2021/03/lab_2_optimization.pdf)
- [метод ньютона](http://fourier.eng.hmc.edu/e176/lectures/ch3/node6.html)

### Одномерные методы поиска

In [2]:
def get_value(fdict, f, x: float):
    if (x not in fdict):
        v = f(x)
        fdict[x] = v
    else:
        v = fdict[x]
    
    return v, fdict

In [3]:
def get_next_interval_base(fdict, f, a: float, b: float, x1: float, x2: float):
    v1, fdict = get_value(fdict, f, x1)
    v2, fdict = get_value(fdict, f, x2)
    
    if (v1 < v2):
        return fdict, a, x2
    elif (v1 > v2):
        return fdict, x1, b
    else:
        return fdict, x1, x2

In [4]:
def fib_search(f, a, b, verbose=False, eps=1e-3):
    fdict = {}
    
    n = 0
    while (fib(n+2) <= (b-a)/eps):
        n += 1
    
    if (verbose):
        print_iter(0, a, b, 1)
    x1 = a + fib(n)*(b-a)/fib(n+2)
    x2 = a + fib(n+1)*(b-a)/fib(n+2)
    step = 1
    
    while (abs(a - b) > eps):
        pred = a-b
        fdict, a, b = get_next_interval_base(fdict, f, a, b, x1, x2)
        if (a == x1):
            x1 = x2
            x2 = a + fib(n-step+2)*(b-a)/fib(n-step+3)
        else:
            x2 = x1
            x1 = a + fib(n-step+1)*(b-a)/fib(n-step+3)
        if (verbose):
            print_iter(step, a, b, (a-b)/pred)
        step += 1
        
    return (a+b)/2, step-1, len(fdict)

### Finding derivative

In [5]:
def get_dfs(func):
    arg_symbols = symbols(inspect.getargspec(func).args)
    sym_func = func(*arg_symbols)

    return [lambdify(arg_symbols, sym_func.diff(a)) for a in arg_symbols]

In [6]:
def calc_df(func, params):
    dfs = get_dfs(func)
    res = []
    for df in dfs:
        res.append(df(*params))
    return res

In [7]:
def calc_inv_hessian(func, params):
    dfs = get_dfs(func)
    n = len(dfs)
    
    h = [[0 for x in range(n)] for y in range(n)] 
    for i in range(len(dfs)):
        h[i] = get_dfs(dfs[i])
    
    for i in range(n):
        for j in range(n):
            h[i][j] = h[i][j](*params)

    inv_h = np.linalg.inv(h)
    
    return inv_h 

### Visualization methods

In [8]:
def print_res(res, step, xs):
    print('Result:', res)
    print('Total iterations: ', step)
    print('Steps: ', xs)

In [9]:
def draw_countors_plot_with_steps(coords, f, classic = True):
    x_min = -5
    y_min = -5
    x_max = 5
    y_max = 5
    delta = 20
    
    
    x = np.linspace(x_min, x_max, delta)
    y = np.linspace(y_min, y_max, delta)
    X, Y = np.meshgrid(x, y)
    
    if (classic):
        z = f
    else:
        z = get_function_from_symmetric_matrix(f) 
        
    Z = z(X, Y)

    contours = plt.contour(X, Y, Z, 3, colors='black')
    plt.clabel(contours, inline=True, fontsize=12)
    plt.imshow(Z, extent=[x_min, x_max, y_min, y_max], origin='lower',
               cmap='RdGy', alpha=0.5)
    plt.colorbar();

    cx, cy = zip(*coords)
    plt.plot(cx, cy, color='black', marker='o')
    #plt.savefig('images/gradient_steps.png')
    plt.show()

### Test data

In [10]:
f = [
    lambda x, y: 100*(y-x)**2 + (1-x)**2,
    lambda x, y: 100*(y-x**2)**2 + (1-x)**2,
]

f_max = lambda x, y: 2*exp**(-((x-1)/2)**2 - (y-1)**2) + 3*exp**(-((x-2)/3)**2 - (((y-3)/2)**2))

## Метод сопряженных направлений

In [11]:
def conjugate_vecs_method(f, x0, N=1000, optimizer=fib_search, eps=1e-5):
    x = np.array(x0)
    step = 0
    xs = [x]
    
    x_prev = x
    w_prev = np.array(-1) * calc_df(f, x)
    p_prev = w_prev
    step += 1
    
    while(abs(np.linalg.norm(w_prev)) > eps or step < N):
        wk = np.array(-1) * calc_df(f, x_prev)
        gammak = (np.linalg.norm(wk)**2)/(np.linalg.norm(w_prev)**2)
        pk = wk + gammak*p_prev
        psi = lambda chi: f(x_prev + chi * p_prev)
        hk, _, _ = optimizer(psi, 0, 1, False, eps)
        xk = x_prev + hk*pk
        
        x_prev = xk
        w_prev = wk
        p_prev = pk
        step += 1
        xs.append(list(x))
        
    
    return x, step, xs

## Метод Ньютона

- функция должна быть дважды дифференцируемой

In [12]:
def newtons_method(f, x, N=20, eps=1e-4):
    step = 0
    xs = [x]
    lr = 1e-3
    
    while (np.linalg.norm(calc_df(f,x)) > eps):
        x -= lr * np.dot(calc_inv_hessian(f,x), calc_df(f,x))
        step += 1
        xs.append(list(x))
    
    return x, step, xs

In [13]:
res, steps, coords = newtons_method(f[0], [2.,2.])

print_res(res, steps, coords)

<ipython-input-5-e9c77eff3d36>:2: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  arg_symbols = symbols(inspect.getargspec(func).args)
<ipython-input-5-e9c77eff3d36>:2: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  arg_symbols = symbols(inspect.getargspec(func).args)


Result: [1.00004998 1.00004998]
Total iterations:  9899
Steps:  [[2.0, 2.0], [1.999, 1.999], [1.9980010000000001, 1.9980010000000001], [1.9970029990000002, 1.9970029990000002], [1.9960059960010001, 1.9960059960010001], [1.9950099900049991, 1.9950099900049991], [1.9940149800149942, 1.9940149800149942], [1.9930209650349793, 1.9930209650349793], [1.9920279440699442, 1.9920279440699442], [1.9910359161258742, 1.9910359161258742], [1.9900448802097483, 1.9900448802097483], [1.9890548353295385, 1.9890548353295385], [1.988065780494209, 1.988065780494209], [1.9870777147137149, 1.9870777147137149], [1.9860906369990012, 1.9860906369990012], [1.9851045463620023, 1.9851045463620023], [1.9841194418156403, 1.9841194418156403], [1.9831353223738246, 1.9831353223738246], [1.982152187051451, 1.982152187051451], [1.9811700348643995, 1.9811700348643995], [1.9801888648295352, 1.9801888648295352], [1.9792086759647056, 1.9792086759647056], [1.9782294672887408, 1.9782294672887408], [1.9772512378214522, 1.977251

## Старый добрый градиентный спуск

In [14]:
def classic_gradient_descent(f, x, eps=1e-4):
    step = 0
    lr = 1e-3
    xs = [x]
    
    while (np.linalg.norm(np.dot(lr, calc_df(f,x))) > eps):
        x -= np.dot(lr, calc_df(f,x))
        step += 1
        xs.append(list(x))
        
    return x, step, xs

In [15]:
res, steps, coords = classic_gradient_descent(f[0], [2.,2.])

print_res(res, steps, coords)

<ipython-input-5-e9c77eff3d36>:2: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  arg_symbols = symbols(inspect.getargspec(func).args)


Result: [1.07070778 1.0710622 ]
Total iterations:  2652
Steps:  [[2.0, 2.0], [1.998, 2.0], [1.996404, 1.9996], [1.995050392, 1.9989608], [1.993842372816, 1.9981787184000002], [1.992721957187168, 1.9973114492832003], [1.991654411692, 1.9963935508639938], [1.9906189307030149, 1.995445723029595], [1.989603051306925, 1.994480364564279], [1.988599307855782, 1.9935049019128082], [1.9876032280514757, 1.9925237831014029], [1.9866121326053583, 1.9915396720914174], [1.9856244162373595, 1.9905541641942055], [1.984639116996254, 1.9895682146028364], [1.983655658283578, 1.98858239508152], [1.9826736943265992, 1.9875970477219316], [1.9816930176170124, 1.9866123770428652], [1.980713503466949, 1.9856285051576947], [1.9797350767981643, 1.9846455048195455], [1.9787576922488441, 1.9836634192152693], [1.9777813222576315, 1.9826822738219843], [1.976805949925987, 1.9817020835091137], [1.9758315647427602, 1.9807228567924884], [1.9748581600232205, 1.9797445983825428], [1.9738857313750384, 1.9787673107106782], 

In [16]:
def quadratic_gradient_descent(Q, x, eps=1e-2):
    step = 0
    lr = 0.5
    coef_crush =0.5
    xs = [x]
    
    while (np.linalg.norm(Q * x) > eps):
        x -= np.linalg.norm(np.eye(len(Q)) - lr*(coef_crush**step)*Q)*np.array(x)
        step += 1
        xs.append(list(x))
        
    return x, step, xs

## Graphics

### Statistics

### Trajectory